This goes through the process of collecting and properly aggregating data for the grant application for funding for the Indigenous American cultural center in Davidson County. This is the only of it's kind in the State of Tennessee, so the State is used as the main geography as the center would solicit work from artists of various kinds from across the state.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
from functools import reduce
pd.options.mode.chained_assignment = None  # default='warn'
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import sys
sys.path.append("../../Functions and Dictionaries/") # Adds higher directory to python modules path
import geodict
namestocommon = geodict.namestocommon
geotogeoid = geodict.geotogeoid
tofullcensus = geodict.tofullcensus
import sqlite3 as sq

In [2]:
#functions
def percent(x, y):
        try:
            return ((x/y)*100)
        except ZeroDivisionError:
            return 0
def percentchange(x, y):
    try:
        return ((x - y)*100/y)
    except ZeroDivisionError:
        return 0
def realchange(x, y):
    return x-y

In [6]:
conn = sq.connect('../../Data-Pipelines/Outputs/CensusBureau.db')
#PL2020
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_PL_2020]', conn)
PL2020 = pd.DataFrame(sql_query)
PL2020.drop(columns = 'Source', inplace = True)
#ACS52021
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_ACS5_2021_1]', conn)
ACS52021_1 = pd.DataFrame(sql_query)
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_ACS5_2021_2]', conn)
ACS52021_2 = pd.DataFrame(sql_query)
ACS52021 = ACS52021_1.merge(ACS52021_2, on = ['NAME', 'GEO_ID', 'Source'])
ACS52021.drop(columns = 'Source', inplace = True)
dfs2021 = [ACS52021, PL2020]
twothousandtwentyone = reduce(lambda  left,right: pd.merge(left,right,on=['NAME', 'GEO_ID'], how='outer'), dfs2021)

In [7]:
data = twothousandtwentyone.loc[(twothousandtwentyone['NAME'] == 'Tennessee')|(twothousandtwentyone['NAME'] == 'US')]

In [8]:
cols = ['NAME', 'HHIncome:Median for Householder Under 65 Over'

        
            ]
data = data[cols]
data

,NAME,HHIncome:Median for Householder Under 65 Over
153,Tennessee,44459.0


In [19]:
cols = ['Employment:Employed Civilian', 'Employment:Unemployed Civilian', 'Employment:Not in Labor Force', 
             'Employment:Employed Civilian American Indian Alaska Native Alone', 'Employment:Unemployed Civilian American Indian Alaska Native Alone', 
             'Employment:Not in Labor Force American Indian Alaska Native Alone', 'Employment:Employed Civilian Native Hawaiian Pacific Islander Alone', 
             'Employment:Unemployed Civilian Native Hawaiian Pacific Islander Alone', 'Employment:Not in Labor Force Native Hawaiian Pacific Islander Alone', 
             'Employment:Employed Civilian Non-Minority', 'Employment:Unemployed Civilian Non-Minority', 'Employment:Not in Labor Force Non-Minority', 
        'Earnings:Median, All Earners' ,'Earnings:Median, All Earners, White Alone' ,'Earnings:Median, All Earners, Black or African American Alone' ,
        'Earnings:Median, All Earners, American Indian Alaska Native Alone' ,'Earnings:Median, All Earners, Asian Alone' ,
        'Earnings:Median, All Earners, Native Hawaiian Pacific Islander Alone' ,'Earnings:Median, All Earners, Some Other Race Alone' ,'Earnings:Median, All Earners, Two or More Races' ,
        'Earnings:Median, All Earners, Non-Minority' ,'Earnings:Median, All Earners, Hispanic or Latino', 'American Indian Alaska Native Alone', 
        'Native Hawaiian Other Pacific Islander Alone'

        
            ]
data = data[cols]

In [20]:
data['Employment:Employed Civilian Group'] = data['Employment:Employed Civilian American Indian Alaska Native Alone'] + data['Employment:Employed Civilian Native Hawaiian Pacific Islander Alone']
data['Employment:Unemployed Civilian Group'] = data['Employment:Unemployed Civilian American Indian Alaska Native Alone'] + data['Employment:Unemployed Civilian Native Hawaiian Pacific Islander Alone']
data['Employment:Civilian Labor Force Group'] = data['Employment:Employed Civilian Group'] + data['Employment:Unemployed Civilian Group']
data['Civilian Unemployment Rate Group'] = (data['Employment:Unemployed Civilian Group']/data['Employment:Civilian Labor Force Group'])*100

In [21]:
data

,Employment:Employed Civilian,Employment:Unemployed Civilian,Employment:Not in Labor Force,Employment:Employed Civilian American Indian Alaska Native Alone,Employment:Unemployed Civilian American Indian Alaska Native Alone,Employment:Not in Labor Force American Indian Alaska Native Alone,Employment:Employed Civilian Native Hawaiian Pacific Islander Alone,Employment:Unemployed Civilian Native Hawaiian Pacific Islander Alone,Employment:Not in Labor Force Native Hawaiian Pacific Islander Alone,Employment:Employed Civilian Non-Minority,Employment:Unemployed Civilian Non-Minority,Employment:Not in Labor Force Non-Minority,"Earnings:Median, All Earners","Earnings:Median, All Earners, White Alone","Earnings:Median, All Earners, Black or African American Alone","Earnings:Median, All Earners, American Indian Alaska Native Alone","Earnings:Median, All Earners, Asian Alone","Earnings:Median, All Earners, Native Hawaiian Pacific Islander Alone","Earnings:Median, All Earners, Some Other Race Alone","Earnings:Median, All Earners, Two or More Races","Earnings:Median, All Earners, Non-Minority","Earnings:Median, All Earners, Hispanic or Latino",American Indian Alaska Native Alone,Native Hawaiian Other Pacific Islander Alone,Employment:Employed Civilian Group,Employment:Unemployed Civilian Group,Employment:Civilian Labor Force Group,Civilian Unemployment Rate Group
153,3201140.0,179568.0,2105502.0,5880.0,469.0,5368.0,2121.0,358.0,676.0,2377207.0,110233.0,1642496.0,34988.0,36820.0,30347.0,32900.0,43189.0,39321.0,27171.0,30738.0,37328.0,27226.0,28044.0,4115.0,8001.0,827.0,8828.0,9.36792


In [22]:
data['Employment:Civilian Labor Force'] = data['Employment:Employed Civilian'] + data['Employment:Unemployed Civilian']
data['Civilian Unemployment Rate'] = (data['Employment:Unemployed Civilian']/data['Employment:Civilian Labor Force'])*100

In [23]:
data['Employment:Civilian Labor Force Non-Minority'] = data['Employment:Employed Civilian Non-Minority'] + data['Employment:Unemployed Civilian Non-Minority']
data['Civilian Unemployment Rate Non-Minority'] = (data['Employment:Unemployed Civilian Non-Minority']/data['Employment:Civilian Labor Force Non-Minority'])*100

In [24]:
#data = data.drop(columns = cols)

In [25]:
data.head()

,Employment:Employed Civilian,Employment:Unemployed Civilian,Employment:Not in Labor Force,Employment:Employed Civilian American Indian Alaska Native Alone,Employment:Unemployed Civilian American Indian Alaska Native Alone,Employment:Not in Labor Force American Indian Alaska Native Alone,Employment:Employed Civilian Native Hawaiian Pacific Islander Alone,Employment:Unemployed Civilian Native Hawaiian Pacific Islander Alone,Employment:Not in Labor Force Native Hawaiian Pacific Islander Alone,Employment:Employed Civilian Non-Minority,Employment:Unemployed Civilian Non-Minority,Employment:Not in Labor Force Non-Minority,"Earnings:Median, All Earners","Earnings:Median, All Earners, White Alone","Earnings:Median, All Earners, Black or African American Alone","Earnings:Median, All Earners, American Indian Alaska Native Alone","Earnings:Median, All Earners, Asian Alone","Earnings:Median, All Earners, Native Hawaiian Pacific Islander Alone","Earnings:Median, All Earners, Some Other Race Alone","Earnings:Median, All Earners, Two or More Races","Earnings:Median, All Earners, Non-Minority","Earnings:Median, All Earners, Hispanic or Latino",American Indian Alaska Native Alone,Native Hawaiian Other Pacific Islander Alone,Employment:Employed Civilian Group,Employment:Unemployed Civilian Group,Employment:Civilian Labor Force Group,Civilian Unemployment Rate Group,Employment:Civilian Labor Force,Civilian Unemployment Rate,Employment:Civilian Labor Force Non-Minority,Civilian Unemployment Rate Non-Minority
153,3201140.0,179568.0,2105502.0,5880.0,469.0,5368.0,2121.0,358.0,676.0,2377207.0,110233.0,1642496.0,34988.0,36820.0,30347.0,32900.0,43189.0,39321.0,27171.0,30738.0,37328.0,27226.0,28044.0,4115.0,8001.0,827.0,8828.0,9.36792,3380708.0,5.31155,2487440.0,4.431584


In [26]:
data.to_csv('employmentbyrace.csv', index = False)

In [60]:
moredata = pd.read_csv('data/ecdreq1.csv')

In [61]:
moredata.head()

,Unnamed: 0,NAME,GEO_ID,pop,raceeth_whitealone_nothispanicorlatino,raceeth_hispanicorlatino,units_allhousing,occupancy_total_series,occupancy_occupiedunits,occupancy_vacantunits,gq_total_series,gq_institutionalized,gq_institutionalized_correctionalfacilityadults,gq_institutionalized_juvenilefacility,gq_institutionalized_nursingorskillednursingfacility,gq_institutionalized_other,gq_noninstitutionalized,gq_noninstitutionalized_collegeoruniversitystudenthousing,gq_noninstitutionalized_militaryquarters,gq_noninstitutionalized_other,raceeth_total_series,raceeth_white_alone,raceeth_blackafricanamerican_alone,raceeth_americanindianalaskanative_alone,raceeth_asian_alone,raceeth_nativehawaiianotherpacificislander_alone,raceeth_someotherrace_alone,raceeth_twoormoreraces,raceeth_tworaces,raceeth_tworaces_white_blackafricanamerican,raceeth_tworaces_white_americanindianalaskanative,raceeth_tworaces_white_asian,raceeth_tworaces_white_nativehawaiianpacificislander,raceeth_tworaces_white_someotherrace,raceeth_tworaces_blackafricanamerican_americanindianalaskanative,raceeth_tworaces_blackafricanamerican_asian,raceeth_tworaces_blackafricanamerican_nativehawaiianpacificislander,raceeth_tworaces_blackafricanamerican_someotherrace,raceeth_tworaces_americanindianalaskanative_asian,raceeth_tworaces_americanindianalaskanative_nativehawaiianpacificislander,raceeth_tworaces_americanindianalaskanative_someotherrace,raceeth_tworaces_asiannativehawaiianpacificislander,raceeth_tworaces_asiansomeotherrace,raceeth_tworaces_nativehawaiianotherpacificislandersomeotherrace,raceeth_threeraces,raceeth_threeraces_white_blackafricanamerican_americanindianalaskanative,raceeth_threeraces_white_blackafricanamerican_asian,raceeth_threeraces_white_blackafricanamerican_nativehawaiianpacificislander,raceeth_threeraces_white_blackafricanamerican_someotherrace,raceeth_threeraces_white_americanindianalaskanative_asian,raceeth_threeraces_white_americanindianalaskanative_nativehawaiianpacificislander,raceeth_threeraces_white_americanindianalaskanative_someotherrace,raceeth_threeraces_white_asian_nativehawaiianpacificislander,raceeth_threeraces_white_asian_someotherrace,raceeth_threeraces_white_nativehawaiianpacificislander_someotherrace,raceeth_threeraces_blackafricanamerican_americanindianalaskanative_asian,raceeth_threeraces_blackafricanamerican_americanindianalaskanative_nativehawaiianpacificislander,raceeth_threeraces_blackafricanamerican_americanindianalaskanative_someotherrace,raceeth_threeraces_blackafricanamerican_asian_nativehawaiianpacificislander,raceeth_threeraces_blackafricanamerican_asian_someotherrace,raceeth_threeraces_blackafricanamerican_nativehawaiianpacificislander_someotherrace,raceeth_threeraces_americanindianalaskanative_asian_nativehawaiianpacificislander,raceeth_threeraces_americanindianalaskanative_asian_someotherrace,raceeth_threeraces_americanindianalaskanative_nativehawaiianpacificislander_someotherrace,raceeth_threeraces_asian_nativehawaiianpacificislander_someotherrace,raceeth_fourraces,raceeth_fourraces_white_blackafricanamerican_americanindianalaskanative_asian,raceeth_fourraces_white_blackafricanamerican_americanindianalaskanative_nativehawaiianpacificislander,raceeth_fourraces_white_blackafricanamerican_americanindianalaskanative_someotherrace,raceeth_fourraces_white_blackafricanamerican_asian_nativehawaiianpacificislander,raceeth_fourraces_white_blackafricanamerican_asian_someotherrace,raceeth_fourraces_white_blackafricanamerican_nativehawaiianpacificislander_someotherrace,raceeth_fourraces_white_americanindianalaskanative_asian_nativehawaiianpacificislander,raceeth_fourraces_white_americanindianalaskanative_asian_someotherrace,raceeth_fourraces_white_americanindianalaskanative_nativehawaiianpacificislander_someotherrace,raceeth_fourraces_white_asian_nativehawaiianpacificislander_someotherrace,raceeth_fourraces_blackafricanamerican_americanindianalaskanative_asian_nativehawaiianpacificislander,raceeth_fourraces_blackafricanamerican_americani

In [62]:
  moredata = moredata.drop(columns = ['Unnamed: 0', 'NAME', 'GEO_ID'])

In [63]:
moredata.head()

,pop,raceeth_whitealone_nothispanicorlatino,raceeth_hispanicorlatino,units_allhousing,occupancy_total_series,occupancy_occupiedunits,occupancy_vacantunits,gq_total_series,gq_institutionalized,gq_institutionalized_correctionalfacilityadults,gq_institutionalized_juvenilefacility,gq_institutionalized_nursingorskillednursingfacility,gq_institutionalized_other,gq_noninstitutionalized,gq_noninstitutionalized_collegeoruniversitystudenthousing,gq_noninstitutionalized_militaryquarters,gq_noninstitutionalized_other,raceeth_total_series,raceeth_white_alone,raceeth_blackafricanamerican_alone,raceeth_americanindianalaskanative_alone,raceeth_asian_alone,raceeth_nativehawaiianotherpacificislander_alone,raceeth_someotherrace_alone,raceeth_twoormoreraces,raceeth_tworaces,raceeth_tworaces_white_blackafricanamerican,raceeth_tworaces_white_americanindianalaskanative,raceeth_tworaces_white_asian,raceeth_tworaces_white_nativehawaiianpacificislander,raceeth_tworaces_white_someotherrace,raceeth_tworaces_blackafricanamerican_americanindianalaskanative,raceeth_tworaces_blackafricanamerican_asian,raceeth_tworaces_blackafricanamerican_nativehawaiianpacificislander,raceeth_tworaces_blackafricanamerican_someotherrace,raceeth_tworaces_americanindianalaskanative_asian,raceeth_tworaces_americanindianalaskanative_nativehawaiianpacificislander,raceeth_tworaces_americanindianalaskanative_someotherrace,raceeth_tworaces_asiannativehawaiianpacificislander,raceeth_tworaces_asiansomeotherrace,raceeth_tworaces_nativehawaiianotherpacificislandersomeotherrace,raceeth_threeraces,raceeth_threeraces_white_blackafricanamerican_americanindianalaskanative,raceeth_threeraces_white_blackafricanamerican_asian,raceeth_threeraces_white_blackafricanamerican_nativehawaiianpacificislander,raceeth_threeraces_white_blackafricanamerican_someotherrace,raceeth_threeraces_white_americanindianalaskanative_asian,raceeth_threeraces_white_americanindianalaskanative_nativehawaiianpacificislander,raceeth_threeraces_white_americanindianalaskanative_someotherrace,raceeth_threeraces_white_asian_nativehawaiianpacificislander,raceeth_threeraces_white_asian_someotherrace,raceeth_threeraces_white_nativehawaiianpacificislander_someotherrace,raceeth_threeraces_blackafricanamerican_americanindianalaskanative_asian,raceeth_threeraces_blackafricanamerican_americanindianalaskanative_nativehawaiianpacificislander,raceeth_threeraces_blackafricanamerican_americanindianalaskanative_someotherrace,raceeth_threeraces_blackafricanamerican_asian_nativehawaiianpacificislander,raceeth_threeraces_blackafricanamerican_asian_someotherrace,raceeth_threeraces_blackafricanamerican_nativehawaiianpacificislander_someotherrace,raceeth_threeraces_americanindianalaskanative_asian_nativehawaiianpacificislander,raceeth_threeraces_americanindianalaskanative_asian_someotherrace,raceeth_threeraces_americanindianalaskanative_nativehawaiianpacificislander_someotherrace,raceeth_threeraces_asian_nativehawaiianpacificislander_someotherrace,raceeth_fourraces,raceeth_fourraces_white_blackafricanamerican_americanindianalaskanative_asian,raceeth_fourraces_white_blackafricanamerican_americanindianalaskanative_nativehawaiianpacificislander,raceeth_fourraces_white_blackafricanamerican_americanindianalaskanative_someotherrace,raceeth_fourraces_white_blackafricanamerican_asian_nativehawaiianpacificislander,raceeth_fourraces_white_blackafricanamerican_asian_someotherrace,raceeth_fourraces_white_blackafricanamerican_nativehawaiianpacificislander_someotherrace,raceeth_fourraces_white_americanindianalaskanative_asian_nativehawaiianpacificislander,raceeth_fourraces_white_americanindianalaskanative_asian_someotherrace,raceeth_fourraces_white_americanindianalaskanative_nativehawaiianpacificislander_someotherrace,raceeth_fourraces_white_asian_nativehawaiianpacificislander_someotherrace,raceeth_fourraces_blackafricanamerican_americanindianalaskanative_asian_nativehawaiianpacificislander,raceeth_fourraces_blackafricanamerican_americanindianalaskanative_asian

In [65]:
cols = ['pop', 
        'raceeth_americanindianalaskanative_alone', 
        'raceeth_nativehawaiianotherpacificislander_alone', 
        'raceeth_tworaces_white_americanindianalaskanative', 
        'raceeth_tworaces_white_nativehawaiianpacificislander', 
        'raceeth_tworaces_blackafricanamerican_americanindianalaskanative', 
        'raceeth_tworaces_blackafricanamerican_nativehawaiianpacificislander', 
        'raceeth_tworaces_americanindianalaskanative_asian', 
        'raceeth_tworaces_americanindianalaskanative_nativehawaiianpacificislander', 
        'raceeth_tworaces_americanindianalaskanative_someotherrace', 
        'raceeth_tworaces_asiannativehawaiianpacificislander', 
        'raceeth_tworaces_nativehawaiianotherpacificislandersomeotherrace', 
        'raceeth_threeraces_white_blackafricanamerican_americanindianalaskanative', 
       'raceeth_tworaces_americanindianalaskanative_someotherrace', 'raceeth_tworaces_asiannativehawaiianpacificislander', 
        'raceeth_tworaces_nativehawaiianotherpacificislandersomeotherrace', 'raceeth_threeraces_white_blackafricanamerican_americanindianalaskanative', 
        'raceeth_threeraces_white_blackafricanamerican_nativehawaiianpacificislander', 'raceeth_threeraces_white_americanindianalaskanative_asian', 
        'raceeth_threeraces_white_americanindianalaskanative_nativehawaiianpacificislander', 'raceeth_threeraces_white_americanindianalaskanative_someotherrace', 
        'raceeth_threeraces_white_asian_nativehawaiianpacificislander', 'raceeth_threeraces_white_asian_someotherrace', 
        'raceeth_threeraces_white_nativehawaiianpacificislander_someotherrace', 'raceeth_threeraces_blackafricanamerican_americanindianalaskanative_asian', 
        'raceeth_threeraces_blackafricanamerican_americanindianalaskanative_nativehawaiianpacificislander', 
        'raceeth_threeraces_blackafricanamerican_americanindianalaskanative_someotherrace', 
        'raceeth_threeraces_blackafricanamerican_asian_nativehawaiianpacificislander', 'raceeth_threeraces_blackafricanamerican_asian_someotherrace', 
        'raceeth_threeraces_blackafricanamerican_nativehawaiianpacificislander_someotherrace', 
        'raceeth_threeraces_americanindianalaskanative_asian_nativehawaiianpacificislander', 'raceeth_threeraces_americanindianalaskanative_asian_someotherrace', 
        'raceeth_threeraces_americanindianalaskanative_nativehawaiianpacificislander_someotherrace', 
        'raceeth_threeraces_asian_nativehawaiianpacificislander_someotherrace', 'raceeth_fourraces_white_blackafricanamerican_americanindianalaskanative_asian',
        'raceeth_fourraces_white_blackafricanamerican_americanindianalaskanative_nativehawaiianpacificislander', 
        'raceeth_fourraces_white_blackafricanamerican_americanindianalaskanative_someotherrace', 
        'raceeth_fourraces_white_blackafricanamerican_asian_nativehawaiianpacificislander',
        'raceeth_fourraces_white_blackafricanamerican_nativehawaiianpacificislander_someotherrace', 
        'raceeth_fourraces_white_americanindianalaskanative_asian_nativehawaiianpacificislander', 
        'raceeth_fourraces_white_americanindianalaskanative_asian_someotherrace', 
        'raceeth_fourraces_white_americanindianalaskanative_nativehawaiianpacificislander_someotherrace', 
        'raceeth_fourraces_white_asian_nativehawaiianpacificislander_someotherrace', 
        'raceeth_fourraces_blackafricanamerican_americanindianalaskanative_asian_nativehawaiianpacificislander', 
        'raceeth_fourraces_blackafricanamerican_americanindianalaskanative_asian_someotherrace', 
        'raceeth_fourraces_blackafricanamerican_americanindianalaskanative_nativehawaiianpacificislander_someotherrace', 
        'raceeth_fourraces_blackafricanamerican_asian_nativehawaiianpacificislander_someotherrace', 
        'raceeth_fourraces_americanindianalaskanative_asian_nativehawaiianpacificislander_someotherrace', 'raceeth_fiveraces', 'raceeth_sixraces'
       ]
moredata = moredata[cols]

In [67]:
ref = moredata[['pop', 
        'raceeth_americanindianalaskanative_alone', 
        'raceeth_nativehawaiianotherpacificislander_alone']]

In [81]:
somenative

0    173154.0
Name: all with some native, dtype: float64

In [83]:
totalpop = ref['pop']
nativealone = ref['raceeth_americanindianalaskanative_alone'] + ref['raceeth_nativehawaiianotherpacificislander_alone']

In [84]:
(nativealone/totalpop)*100

0    0.465341
dtype: float64

In [85]:
nativealone

0    32159.0
dtype: float64

In [86]:
totalpop

0    6910840.0
Name: pop, dtype: float64

In [69]:
moredata = moredata.drop(columns = ['pop', 
        'raceeth_americanindianalaskanative_alone', 
        'raceeth_nativehawaiianotherpacificislander_alone'])

In [70]:
moredata.head()

,raceeth_tworaces_white_americanindianalaskanative,raceeth_tworaces_white_nativehawaiianpacificislander,raceeth_tworaces_blackafricanamerican_americanindianalaskanative,raceeth_tworaces_blackafricanamerican_nativehawaiianpacificislander,raceeth_tworaces_americanindianalaskanative_asian,raceeth_tworaces_americanindianalaskanative_nativehawaiianpacificislander,raceeth_tworaces_americanindianalaskanative_someotherrace,raceeth_tworaces_asiannativehawaiianpacificislander,raceeth_tworaces_nativehawaiianotherpacificislandersomeotherrace,raceeth_threeraces_white_blackafricanamerican_americanindianalaskanative,raceeth_tworaces_americanindianalaskanative_someotherrace,raceeth_tworaces_asiannativehawaiianpacificislander,raceeth_tworaces_nativehawaiianotherpacificislandersomeotherrace,raceeth_threeraces_white_blackafricanamerican_americanindianalaskanative,raceeth_threeraces_white_blackafricanamerican_nativehawaiianpacificislander,raceeth_threeraces_white_americanindianalaskanative_asian,raceeth_threeraces_white_americanindianalaskanative_nativehawaiianpacificislander,raceeth_threeraces_white_americanindianalaskanative_someotherrace,raceeth_threeraces_white_asian_nativehawaiianpacificislander,raceeth_threeraces_white_asian_someotherrace,raceeth_threeraces_white_nativehawaiianpacificislander_someotherrace,raceeth_threeraces_blackafricanamerican_americanindianalaskanative_asian,raceeth_threeraces_blackafricanamerican_americanindianalaskanative_nativehawaiianpacificislander,raceeth_threeraces_blackafricanamerican_americanindianalaskanative_someotherrace,raceeth_threeraces_blackafricanamerican_asian_nativehawaiianpacificislander,raceeth_threeraces_blackafricanamerican_asian_someotherrace,raceeth_threeraces_blackafricanamerican_nativehawaiianpacificislander_someotherrace,raceeth_threeraces_americanindianalaskanative_asian_nativehawaiianpacificislander,raceeth_threeraces_americanindianalaskanative_asian_someotherrace,raceeth_threeraces_americanindianalaskanative_nativehawaiianpacificislander_someotherrace,raceeth_threeraces_asian_nativehawaiianpacificislander_someotherrace,raceeth_fourraces_white_blackafricanamerican_americanindianalaskanative_asian,raceeth_fourraces_white_blackafricanamerican_americanindianalaskanative_nativehawaiianpacificislander,raceeth_fourraces_white_blackafricanamerican_americanindianalaskanative_someotherrace,raceeth_fourraces_white_blackafricanamerican_asian_nativehawaiianpacificislander,raceeth_fourraces_white_blackafricanamerican_nativehawaiianpacificislander_someotherrace,raceeth_fourraces_white_americanindianalaskanative_asian_nativehawaiianpacificislander,raceeth_fourraces_white_americanindianalaskanative_asian_someotherrace,raceeth_fourraces_white_americanindianalaskanative_nativehawaiianpacificislander_someotherrace,raceeth_fourraces_white_asian_nativehawaiianpacificislander_someotherrace,raceeth_fourraces_blackafricanamerican_americanindianalaskanative_asian_nativehawaiianpacificislander,raceeth_fourraces_blackafricanamerican_americanindianalaskanative_asian_someotherrace,raceeth_fourraces_blackafricanamerican_americanindianalaskanative_nativehawaiianpacificislander_someotherrace,raceeth_fourraces_blackafricanamerican_asian_nativehawaiianpacificislander_someotherrace,raceeth_fourraces_americanindianalaskanative_asian_nativehawaiianpacificislander_someotherrace,raceeth_fiveraces,raceeth_sixraces
0,109625.0,2453.0,6326.0,534.0,347.0,104.0,2801.0,926.0,341.0,6873.0,2801.0,926.0,341.0,6873.0,261.0,1380.0,210.0,3407.0,1046.0,777.0,132.0,140.0,49.0,250.0,166.0,67.0,32.0,30.0,12.0,6.0,47.0,466.0,82.0,770.0,79.0,12.0,112.0,83.0,16.0,36.0,25.0,6.0,8.0,16.0,8.0,250.0,20.0


In [76]:
moredata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 47 columns):
 #   Column                                                                                                         Non-Null Count  Dtype  
---  ------                                                                                                         --------------  -----  
 0   raceeth_tworaces_white_americanindianalaskanative                                                              1 non-null      float64
 1   raceeth_tworaces_white_nativehawaiianpacificislander                                                           1 non-null      float64
 2   raceeth_tworaces_blackafricanamerican_americanindianalaskanative                                               1 non-null      float64
 3   raceeth_tworaces_blackafricanamerican_nativehawaiianpacificislander                                            1 non-null      float64
 4   raceeth_tworaces_americanindianalaskanative_asian     

In [74]:
cols = moredata.columns

In [77]:
cols

Index(['raceeth_tworaces_white_americanindianalaskanative', 'raceeth_tworaces_white_nativehawaiianpacificislander',
       'raceeth_tworaces_blackafricanamerican_americanindianalaskanative', 'raceeth_tworaces_blackafricanamerican_nativehawaiianpacificislander',
       'raceeth_tworaces_americanindianalaskanative_asian', 'raceeth_tworaces_americanindianalaskanative_nativehawaiianpacificislander',
       'raceeth_tworaces_americanindianalaskanative_someotherrace', 'raceeth_tworaces_asiannativehawaiianpacificislander',
       'raceeth_tworaces_nativehawaiianotherpacificislandersomeotherrace',
       'raceeth_threeraces_white_blackafricanamerican_americanindianalaskanative', 'raceeth_tworaces_americanindianalaskanative_someotherrace',
       'raceeth_tworaces_asiannativehawaiianpacificislander', 'raceeth_tworaces_nativehawaiianotherpacificislandersomeotherrace',
       'raceeth_threeraces_white_blackafricanamerican_americanindianalaskanative',
       'raceeth_threeraces_white_blackafricana

In [78]:
moredata['all with some native'] = moredata[cols].sum(axis = 1)

In [80]:
somenative = moredata['all with some native'] 